In [1]:
%load_ext autoreload
%autoreload 2
%env TF_FORCE_GPU_ALLOW_GROWTH=true

from usl_models.flood_ml import model_params
from usl_models.flood_ml import model
from usl_models.flood_ml import fake_data
from usl_models.flood_ml import constants


model_dir = "gs://usl_models_bucket/flood_model"
lstm_units = 128
lstm_kernel_size = 3
lstm_dropout = 0.2
lstm_recurrent_dropout = 0.2
epochs = 1

config = fake_data.DatasetConfig(width = 1000, height = 1000, batch_size=1, storm_duration=4)
# Create FloodModelParams from hyperparameters
params = model_params.FloodModelParams(
    batch_size=config.batch_size,
    lstm_units=lstm_units,
    lstm_kernel_size=lstm_kernel_size,
    lstm_dropout=lstm_dropout,
    lstm_recurrent_dropout=lstm_recurrent_dropout,
    epochs=epochs,
)
model_history = []

flood_model = model.FloodModel(params, spatial_dims=(config.height, config.width))

dataset, storm_duration = fake_data.get_fake_dataset(config)
flood_model._model.fit(dataset)

env: TF_FORCE_GPU_ALLOW_GROWTH=true


2024-07-04 14:59:48.148404: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 14:59:48.148461: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 14:59:48.149149: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-04 14:59:48.153661: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-04 14:59:48.824135: W tensorflow/compiler/tf2

st_input shape: Tensor("flood_conv_lstm/Shape:0", shape=(5,), dtype=int32)
geo_input shape: Tensor("flood_conv_lstm/Shape_1:0", shape=(4,), dtype=int32)
temp_input shape: Tensor("flood_conv_lstm/Shape_2:0", shape=(3,), dtype=int32)
st_input shape: Tensor("flood_conv_lstm/Shape:0", shape=(5,), dtype=int32)
geo_input shape: Tensor("flood_conv_lstm/Shape_1:0", shape=(4,), dtype=int32)
temp_input shape: Tensor("flood_conv_lstm/Shape_2:0", shape=(3,), dtype=int32)

---------------------------------------------------------
Generate sample 0.

---------------------------------------------------------
Generate sample 1.


2024-07-04 14:59:52.938301: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-07-04 14:59:56.434936: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f7202df49a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-04 14:59:56.434985: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-07-04 14:59:56.445318: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1720130396.569367    4826 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


      1/Unknown - 9s 9s/step - loss: 1.0002 - mean_absolute_error: 0.7980 - root_mean_squared_error: 1.0001
---------------------------------------------------------
Generate sample 2.
      2/Unknown - 9s 72ms/step - loss: 1.0002 - mean_absolute_error: 0.7979 - root_mean_squared_error: 1.0001
---------------------------------------------------------
Generate sample 3.
      3/Unknown - 9s 69ms/step - loss: 1.0000 - mean_absolute_error: 0.7979 - root_mean_squared_error: 1.0000
---------------------------------------------------------
Generate sample 4.
      4/Unknown - 9s 67ms/step - loss: 0.9998 - mean_absolute_error: 0.7978 - root_mean_squared_error: 0.9999
---------------------------------------------------------
Generate sample 5.
      5/Unknown - 9s 66ms/step - loss: 1.0000 - mean_absolute_error: 0.7979 - root_mean_squared_error: 1.0000
---------------------------------------------------------
Generate sample 6.
      6/Unknown - 9s 66ms/step - loss: 1.0000 - mean_absolute_error